<a href="https://colab.research.google.com/github/pointmina/practicalNLP/blob/main/model_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 30.6 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset   # Huggingface 데이터셋 패키지 import
data = load_dataset("sepidmnorozy/Korean_sentiment")    # 데이터 다운로드

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/sepidmnorozy___csv/sepidmnorozy--Korean_sentiment-c4fbcaaf8f2ffcfa/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
train_data = data['train']
dev_data = data['validation']
test_data = data['test']

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [9]:
#CountVectorizer 수정, N-gram(1,1), TF-IDF, 형태소분석 안함, 불용어 처리 안함

vectorizer = TfidfVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,1))
vectorizer.fit(train_data['text'])

#print("Vectorizer vocab: ", vectorizer.vocabulary_)
print("Vectorizer vocab size: ", len(vectorizer.vocabulary_))

train_vectors = vectorizer.transform(train_data['text'])
dev_vectors = vectorizer.transform(dev_data['text'])
test_vectors = vectorizer.transform(test_data['text'])

# print(train_vectors)

Vectorizer vocab size:  100494


In [10]:
sample_num = 0
sample_origin = train_data[sample_num]
sample_transform = train_vectors[sample_num]
sample_inverse_transform = vectorizer.inverse_transform(sample_transform)
print("Original Input:{}\nTransformed: {}\nInv-transformed: {}".format(sample_origin, sample_transform, sample_inverse_transform))

Original Input:{'label': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}
Transformed:   (0, 82955)	0.24803531244759022
  (0, 76600)	0.3586563080553068
  (0, 76567)	0.26583172003536687
  (0, 70669)	0.3653684561951893
  (0, 61807)	0.22189503499966104
  (0, 59095)	0.31451462517036477
  (0, 41839)	0.37816376511131566
  (0, 41511)	0.24536839594296717
  (0, 32878)	0.43386436732096745
  (0, 32639)	0.2570503765929463
Inv-transformed: [array(['지금', '재밌었고', '재밌어요', '이상으로', '역시',
       '어렸을때', '봤을때도', '봐도', '몇억배', '명작'],
      dtype='<U338')]


In [11]:
train_vectors = vectorizer.transform(train_data['text'])
dev_vectors = vectorizer.transform(dev_data['text'])
test_vectors = vectorizer.transform(test_data['text'])

In [12]:
##### 훈련세트에서 훈련하고 평가하기
from sklearn.svm import LinearSVC     #Linear kernel SVM을 사용하기 위한 클래스     

svm = LinearSVC()     # 모델 정의
svm.fit(train_vectors, train_data['label'])     # 모델 학습

print(svm.coef_)      # weights (w)
print(svm.intercept_)     # bias (b)

[[-0.31194654 -0.33987584  0.17500253 ... -0.34580569  0.11423289
  -0.0032787 ]]
[-0.07857297]


In [13]:
from sklearn.metrics import accuracy_score    # Accuracy 측정 함수 import

final_model = svm
pred_results = final_model.predict(test_vectors)    # 최종 모델로 test 데이터 예측
accuracy = accuracy_score(test_data['label'], pred_results)   # 정확도 측정

print("Accuracy: {:.2f}%".format(accuracy*100))   # 정확도 출력 

Accuracy: 80.88%
